In [1]:
%load_ext autoreload
%autoreload 2
from hereutil import here, add_to_sys_path
add_to_sys_path(here())

from src.common_basis import *

eng, con = get_connection()

In [2]:
import pandas as pd
d = pd.read_sql("""SELECT `post_text`, `post_id`
FROM `dhh23`.`incel_posts_cleaned_a`
WHERE (`post_text` REGEXP '\\\\b(I|he|she|you|we|they)\\\\b')""", con)
d

,post_text,post_id
0,Or maybe she just realized Chad will never com...,1
1,Can you PM me some of the names of the accounts?,9
2,\nWhy does she do it? What is the endgame for ...,7
3,"This is not a joke, I'm 100% serious \n \nSinc...",8
4,bitch is paler than I am,16
...,...,...
1000891,\nAdmins don't give a fuck didn't you hear thi...,2266021
1000892,\nThat's because this site IS Lookism 2.0. It ...,2266023
1000893,How am I delusional? What is your explanation ...,2266007
1000894,\nThe desperation caused by inceldom can lead ...,2266008


In [3]:
import spacy
spacy.require_gpu()
import re

from tqdm.auto import tqdm
nlp = spacy.load("en_core_web_sm", disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer", "ner"])
nlp.enable_pipe("senter")
matching_sentences = []
for doc, context in tqdm(nlp.pipe(map(lambda t: (re.sub("\\.?\\s*\\n+\\s*",". ",re.sub("Click to expand...",".", t[0])), t[1]), d.itertuples(index=False, name=None)), as_tuples=True, n_process=1), total=len(d), smoothing=0):
    for sentence in doc.sents:
        if re.search("\\b(I|he|she|you|we|they)\\b", sentence.text) is not None:
            matching_sentences.append((sentence.text, context))
len(matching_sentences)
import pickle
with here("data/output/jiemakel/pp_sentences.pickle").open('wb') as f:
    pickle.dump(matching_sentences, f)

  0%|          | 0/1000896 [00:00<?, ?it/s]

In [4]:
import spacy
spacy.require_gpu()
from tqdm.auto import tqdm
nlp = spacy.load("en_core_web_trf", disable=["ner"])
from spacy.language import Language

@Language.component("remove_trf_data")
def remove_trf_data(doc):
    doc._.trf_data = None
    return doc

nlp.add_pipe("remove_trf_data")
import pickle
with here("data/output/jiemakel/pp_sentences.pickle").open('rb') as f:
    matching_sentences = pickle.load(f)
sentences = [sentence[0] for sentence in matching_sentences]
docs = list(tqdm(nlp.pipe(sentences, n_process=1, batch_size=256), total=len(sentences), smoothing=0))

  0%|          | 0/1391693 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (550 > 512). Running this sequence through the model will result in indexing errors


In [6]:
from spacy.tokens import DocBin
doc_bin = DocBin(docs=docs)
nlp.vocab.to_disk(here('data/output/jiemakel/pp_sentences.vocab.txt'))
doc_bin.to_disk(here("data/output/jiemakel/pp_sentences.spacydocs"))

In [10]:
import spacy
from spacy.tokens import DocBin
nlp_vocab = spacy.vocab.Vocab().from_disk(here('data/output/jiemakel/pp_sentences.vocab.txt'))
docs2 = list(DocBin().from_disk(here("data/output/jiemakel/pp_sentences.spacydocs")).get_docs(nlp_vocab))
docs2[1][0].pos_

'AUX'

In [8]:
for token in docs[99010]:
    if token.text=="are" or token.text=="'re":
        if token.dep_ == 'ROOT':
            print(list(map(lambda t: list(t.subtree), token.rights)))
        elif 'VerbForm=Part' in token.head.morph:
            print(token.head, list(next(token.head.rights).subtree))
        else:
            print(list(token.subtree))
#        print(list(map(lambda t: list(t.subtree), token.head.rights)))
        #for ot in token.subtree:
#            print(ot, ot.pos_)
 #           if ot.pos_=="ADJ" or ot.pos_=="VERB":
 #               print(ot)
        print(list(map(lambda t: list(t.subtree), token.head.lefts)),token.head,list(map(lambda t: list(t.subtree), token.head.rights)))

In [361]:
from spacy import displacy
displacy.serve(docs[61][0], style='dep')




Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


'Life is good if you are not a mentalcel.'

In [ ]:
d['post_content']